In [13]:
import requests

url = 'http://apis.data.go.kr/B552657/AEDInfoInqireService/getAedFullDown'
api_key = 'csNyrGeiYsO1Dw36uAgMBU56sZAkPedxMNwB9u1bqYgGaITRh%2Bsn6MAWl3QfCBCQBqZEgGGLsJKeRr%2FN8l7GkQ%3D%3D'
qs = 'serviceKey={0}&pageNo={1}&numOfRows={2}'.format(api_key, 1, 10)
response = requests.get(url + "?" + qs)

In [3]:
# response.content

In [14]:
import xmltodict, json
response_dict = xmltodict.parse(response.content)

In [6]:
# response_dict

In [15]:
response_json = json.dumps(response_dict, ensure_ascii=False)

In [16]:
response_json

'{"response": {"header": {"resultCode": "00", "resultMsg": "NORMAL SERVICE."}, "body": {"items": {"item": [{"buildAddress": "부산광역시 사하구 낙동대로398번길 12 사하구청의회1층 의원사무실입구", "buildPlace": "사하구청의회1층 의원사무실입구", "clerkTel": "051-220-4871", "manager": "김은아", "managerTel": "0512204871", "mfg": "(주)나눔테크", "model": "NT-381.C", "org": "사하구청의회", "rnum": "1", "wgs84Lat": "35.1045007076", "wgs84Lon": "128.9747894929", "zipcode1": "493", "zipcode2": "28"}, {"buildAddress": "전라북도 남원시 주천면 정령치로 255", "buildPlace": "지리산북부사무소 1층 복도", "clerkTel": "063-630-8943", "manager": "김재용", "managerTel": "010-***-****", "mfg": "씨유메디칼", "model": "CU-SP1", "org": "국립공원관리공단지리산국립공원북부사무소", "rnum": "2", "wgs84Lat": "35.3896423449", "wgs84Lon": "127.4614630454", "zipcode1": "558", "zipcode2": "02"}, {"buildAddress": "서울특별시 중구 소공로 106 신세계 조선호텔", "buildPlace": "보안실", "clerkTel": "023170019", "manager": "김현조", "managerTel": "02-317-0019", "mfg": "니혼코텐코리아", "model": "AED-2100k", "org": "조선호텔", "rnum": "3", "wgs84Lat": "37.5644836818

In [17]:
response_dict2 = json.loads(response_json)

In [19]:
# type(response_dict2)

In [20]:
response_dict2['response'].keys()
values = response_dict2['response']['body']['items']['item']

In [29]:
# type(values)
# len(values)
values[9]

{'buildAddress': '대전광역시 서구 정림서로 162-18',
 'buildPlace': '관리동 1층',
 'clerkTel': '042-583-1188',
 'manager': '장극순',
 'managerTel': '010-***-****',
 'mfg': '(주)씨유메디칼시스템',
 'model': 'i-padNF1200',
 'org': '강변들보람아파트',
 'rnum': '10',
 'wgs84Lat': '36.3005599077',
 'wgs84Lon': '127.3645882629',
 'zipcode1': '353',
 'zipcode2': '98'}

In [37]:
columns = [
    'num', 'Address', 'DetailedAdress'
]

keys = [
    'rnum', 'buildAddress', 'buildPlace'
]

key_column_map = { key: column for key, column in zip(keys, columns) }
print( key_column_map )
column_key_map = { column: key for key, column in zip(keys, columns) }
print( column_key_map )

len(keys), len(columns)

{'rnum': 'num', 'buildAddress': 'Address', 'buildPlace': 'DetailedAdress'}
{'num': 'rnum', 'Address': 'buildAddress', 'DetailedAdress': 'buildPlace'}


(3, 3)

In [39]:
import pymysql

connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'Rjwu8253', 'db': 'demodb', 'charset': 'utf8' }

conn = pymysql.connect(**connection_info)
cursor = conn.cursor()

for value in values:

    sql = "SELECT IFNULL(COUNT(*), 0) FROM aedlocation WHERE num = %s and Address = %s and DetailedAddress = %s"
    cursor.execute(sql, (value['rnum'], value['buildAddress'], value['buildPlace']))
    if cursor.fetchone()[0] > 0:
        continue    

    sql = 'INSERT INTO aedlocation VALUES(%s, %s, %s)'
    cursor.execute(sql, [ value[column_key_map[column]] for column in columns[0:] ])

conn.commit()

conn.close()